In [80]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import scipy.spatial.distance as dist
import geopandas as gpd
from operator import itemgetter
import networkx as nx
import csv

In [191]:
# cargo bike route
bike = gpd.read_file('./OSM_point_bike_deport.geojson')

In [195]:
bike.head()

,Geographic Area Name,Population,Demand,OSM_lon,OSM_lat,geometry
0,"Block 1002, Block Group 1, Census Tract 39, Ki...",575,0,-73.980702,40.685324,POINT (-73.98070 40.68532)
1,"Block 1000, Block Group 1, Census Tract 1, Kin...",25,1,-73.993838,40.702615,POINT (-73.99384 40.70262)
2,"Block 1001, Block Group 1, Census Tract 1, Kin...",36,1,-73.994421,40.702830,POINT (-73.99442 40.70283)
3,"Block 1004, Block Group 1, Census Tract 1, Kin...",14,1,-73.993367,40.701572,POINT (-73.99337 40.70157)
4,"Block 1005, Block Group 1, Census Tract 1, Kin...",182,4,-73.993300,40.702408,POINT (-73.99330 40.70241)


In [236]:
bike.Demand.sum()

6417.0

In [197]:
len(bike)

812

In [201]:
clients = 811                                                  # 客户数量
capacity = 40                                              # 车辆载重                                        # 车辆一次行驶的最大距离                                             # 各个客户及配送中心距离
demand = bike.Demand.values                                # 811个客户分布需要的货物的需求量，第0位为配送中心
savings = []                                               # 节约度
Routes = []                                                # 路线
Cost = 0  

In [203]:
# bike distance matrix
distance = pd.read_csv('./Dis_Bike.csv',index_col=0)

In [176]:
distance = distance.reset_index(drop=True)
distance.head()

,0,1,2,3,4,5,6,7,8,9,...,802,803,804,805,806,807,808,809,810,811
0,0.000,2695.145,2749.844,2941.212,2644.312,3226.907,3226.907,2699.218,3226.907,2622.211,...,3792.248,4012.269,4097.211,4091.304,3927.412,4097.211,4353.956,4787.182,3179.740,3252.390
1,2725.616,0.000,54.699,246.067,68.294,2934.311,2934.311,402.066,2934.311,630.631,...,3646.352,3866.373,3951.315,3945.408,3781.516,3951.315,4208.060,4641.286,3364.450,3437.100
2,2781.341,174.852,0.000,420.919,124.019,2879.612,2879.612,576.918,2879.612,766.864,...,3702.077,3922.098,4007.040,4001.133,3837.241,4007.040,4263.785,4697.011,3420.175,3492.825
3,2656.857,208.515,263.214,0.000,157.682,3142.826,3142.826,610.581,3142.826,642.380,...,3657.314,3877.335,3962.277,3956.370,3792.478,3962.277,4219.022,4652.248,3372.586,3445.236
4,2669.382,50.833,105.532,296.900,0.000,2985.144,2985.144,452.899,2985.144,654.905,...,3590.118,3810.139,3895.081,3889.174,3725.282,3895.081,4151.826,4585.052,3308.216,3380.866


In [213]:
distance

,0,1,2,3,4,5,6,7,8,9,...,802,803,804,805,806,807,808,809,810,811
0,0.000,2695.145,2749.844,2941.212,2644.312,3226.907,3226.907,2699.218,3226.907,2622.211,...,3792.248,4012.269,4097.211,4091.304,3927.412,4097.211,4353.956,4787.182,3179.740,3252.390
1,2725.616,0.000,54.699,246.067,68.294,2934.311,2934.311,402.066,2934.311,630.631,...,3646.352,3866.373,3951.315,3945.408,3781.516,3951.315,4208.060,4641.286,3364.450,3437.100
2,2781.341,174.852,0.000,420.919,124.019,2879.612,2879.612,576.918,2879.612,766.864,...,3702.077,3922.098,4007.040,4001.133,3837.241,4007.040,4263.785,4697.011,3420.175,3492.825
3,2656.857,208.515,263.214,0.000,157.682,3142.826,3142.826,610.581,3142.826,642.380,...,3657.314,3877.335,3962.277,3956.370,3792.478,3962.277,4219.022,4652.248,3372.586,3445.236
4,2669.382,50.833,105.532,296.900,0.000,2985.144,2985.144,452.899,2985.144,654.905,...,3590.118,3810.139,3895.081,3889.174,3725.282,3895.081,4151.826,4585.052,3308.216,3380.866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,3872.481,3982.953,4037.652,4229.020,3932.120,6294.116,6294.116,4284.625,6294.116,4030.674,...,341.823,412.337,50.000,171.499,327.480,0.000,256.745,689.971,1268.884,1341.534
808,4129.226,4239.698,4294.397,4485.765,4188.865,6550.861,6550.861,4541.370,6550.861,4287.419,...,598.568,669.082,256.745,428.244,584.225,256.745,0.000,433.226,1525.629,1598.279
809,4278.542,4389.014,4443.713,4635.081,4338.181,6700.177,6700.177,4690.686,6700.177,4436.735,...,872.510,818.398,530.687,570.245,733.541,530.687,273.942,0.000,1674.945,1747.595
810,2740.436,3294.917,3349.616,3540.984,3244.084,5280.142,5280.142,3444.043,5280.142,3342.638,...,962.222,1198.210,1283.152,1277.245,1113.353,1283.152,1539.897,1973.123,0.000,72.650


In [219]:
distance.columns = distance.columns.astype('int')

In [222]:
# itter 0 routes
Routes=[]
for i in range(1, len(demand)):
    Routes.append([i])

# caculate the saving, neglect deport
savings = []
for i in range(1, len(Routes)+1):
    for j in range(1, len(Routes)+1):
        if i==j:
            pass
        else:
            saving = (distance[i][0] + distance[0][j]) - distance[i][j]
            savings.append([i, j, saving])
savings = sorted(savings, key=itemgetter(2), reverse=True)

In [181]:
savings

[[461, 570, 8996.197],
 [570, 461, 8996.197],
 [301, 302, 8602.496],
 [302, 301, 8602.496],
 [809, 808, 8483.181999999999],
 [808, 809, 8358.556],
 [290, 301, 8335.565999999999],
 [290, 302, 8335.565999999999],
 [301, 290, 8335.565999999999],
 [302, 290, 8335.565999999999],
 [305, 292, 8314.06],
 [301, 292, 8313.211],
 [302, 292, 8313.211],
 [305, 301, 8311.547999999999],
 [305, 302, 8311.547999999999],
 [307, 305, 8205.411],
 [304, 305, 8199.292],
 [304, 307, 8199.292],
 [305, 307, 8195.245],
 [289, 290, 8176.748],
 [289, 301, 8176.748],
 [289, 302, 8176.748],
 [290, 289, 8176.748],
 [301, 289, 8176.748],
 [302, 289, 8176.748],
 [290, 292, 8155.032999999999],
 [307, 304, 8140.339999999998],
 [305, 304, 8130.174],
 [306, 292, 8047.287],
 [306, 305, 8047.287],
 [306, 307, 8047.287],
 [307, 292, 8047.287],
 [307, 306, 8047.287],
 [307, 308, 8047.287],
 [308, 292, 8047.287],
 [308, 305, 8047.287],
 [308, 307, 8047.287],
 [306, 301, 8044.775],
 [306, 302, 8044.775],
 [307, 301, 8044.775],


In [225]:

for i in range(len(savings)):
            startRoute = []
            endRoute = []
            routeDemand = 0
            for j in range(len(Routes)):
                if (savings[i][0] == Routes[j][-1]):
                    endRoute = Routes[j]
                elif (savings[i][1] == Routes[j][0]):
                    startRoute = Routes[j]
                
                if ((len(startRoute) != 0) and (len(endRoute) != 0)):
                    for k in range(len(startRoute)):
                        routeDemand += demand[startRoute[k]]
                    for k in range(len(endRoute)):
                        routeDemand += demand[endRoute[k]]
                    routeDistance = 0
                    routestore = [0]+endRoute+startRoute+[0]
                    for i in range(len(routestore)-1):
                        # print(routestore[i],routestore[i+1])
                        # print(self.distance[routestore[i]][routestore[i+1]])
                        routeDistance += distance[routestore[i]][routestore[i+1]]

                    #print(routestore,"== ==:",routeDistance)

                    if (routeDemand <= capacity):     # 按照限制规则对​​路线进行更改
                        Routes.remove(startRoute)
                        Routes.remove(endRoute)
                        Routes.append(endRoute + startRoute)
                    break
            

In [226]:
# insert deport
for i in range(len(Routes)):
    Routes[i].insert(0,0)
    Routes[i].insert(len(Routes[i]),0)

In [228]:
len(Routes)

170

In [229]:
dd=[]
df=pd.DataFrame()
for i in Routes:
            costs = 0
            load = 0
            for j in range(len(i)-1):
                
                load += demand[i[j]]
                costs += distance[i[j]][i[j+1]]
                
            df = df.append(pd.DataFrame({'Route':[i],'Load':[load],'distance':[costs]}))
                
            print ('Route:',i)
            print ('Load:',load)
            print ('Distance:',costs)
            print('---'*10)
            dd.append(costs)
Cost = 0
for i in range(len(Routes)):
            for j in range(len(Routes[i]) - 1):
                Cost += distance[Routes[i][j]][Routes[i][j + 1]]
            
print('Total Distance:', Cost)

Route: [0, 35, 0]
Load: 36
Distance: 4206.126
------------------------------
Route: [0, 159, 0]
Load: 39
Distance: 2369.804
------------------------------
Route: [0, 187, 0]
Load: 28
Distance: 850.081
------------------------------
Route: [0, 188, 0]
Load: 26
Distance: 1099.835
------------------------------
Route: [0, 803, 805, 807, 802, 0]
Load: 40
Distance: 8013.867000000001
------------------------------
Route: [0, 318, 315, 314, 310, 309, 313, 312, 311, 299, 304, 307, 305, 292, 291, 294, 288, 289, 290, 301, 302, 293, 296, 0]
Load: 40
Distance: 11371.487999999998
------------------------------
Route: [0, 332, 323, 326, 297, 441, 298, 295, 0]
Load: 39
Distance: 8094.998
------------------------------
Route: [0, 576, 652, 575, 0]
Load: 38
Distance: 6594.075000000001
------------------------------
Route: [0, 647, 648, 651, 0]
Load: 40
Distance: 6598.025
------------------------------
Route: [0, 285, 330, 329, 320, 321, 322, 325, 300, 306, 308, 303, 324, 319, 0]
Load: 39
Distance: 8521

In [230]:
df

,Route,Load,distance
0,"[0, 35, 0]",36,4206.126
0,"[0, 159, 0]",39,2369.804
0,"[0, 187, 0]",28,850.081
0,"[0, 188, 0]",26,1099.835
0,"[0, 803, 805, 807, 802, 0]",40,8013.867
...,...,...,...
0,"[0, 202, 378, 241, 372, 232, 0]",40,2764.729
0,"[0, 511, 504, 520, 198, 0]",35,1473.320
0,"[0, 172, 194, 199, 0]",33,955.666
0,"[0, 195, 196, 490, 197, 0]",37,635.268


In [231]:
df.to_csv('./CW_Bike_Route_1219.csv')

In [242]:
# cargo van route
van = gpd.read_file('./OSM_point_van_deport.geojson')

In [243]:
van.head()

,Geographic Area Name,Population,Demand,OSM_lon,OSM_lat,geometry
0,UPS,NaN,0.0,-73.901651,40.653752,POINT (-73.90165 40.65375)
1,"Block 1000, Block Group 1, Census Tract 1, Kin...",25.0,1.0,-73.993838,40.702615,POINT (-73.99384 40.70262)
2,"Block 1001, Block Group 1, Census Tract 1, Kin...",36.0,1.0,-73.994421,40.702830,POINT (-73.99442 40.70283)
3,"Block 1004, Block Group 1, Census Tract 1, Kin...",14.0,1.0,-73.993367,40.701572,POINT (-73.99337 40.70157)
4,"Block 1005, Block Group 1, Census Tract 1, Kin...",182.0,4.0,-73.993300,40.702408,POINT (-73.99330 40.70241)


In [244]:
van.Demand.sum()

6417.0

In [245]:
# cargo van distance matrix
distance = pd.read_csv('./Dis_Van.csv',index_col=0)
distance.head()

,0,1,2,3,4,5,6,7,8,9,...,802,803,804,805,806,807,808,809,810,811
0,0.000,11421.904,11476.603,11667.971,11371.071,12039.825,12039.825,11470.023,12039.825,11393.016,...,9408.190,9666.532,9750.013,9756.582,9581.675,9750.013,9800.901,10051.656,8890.017,8817.367
1,11423.985,0.000,54.699,246.067,68.294,2934.311,2934.311,402.066,2934.311,630.631,...,3646.352,3866.373,3951.315,3945.408,3781.516,3951.315,4208.060,4641.286,3364.450,3437.100
2,11479.710,174.852,0.000,420.919,124.019,2879.612,2879.612,576.918,2879.612,766.864,...,3702.077,3922.098,4007.040,4001.133,3837.241,4007.040,4263.785,4697.011,3420.175,3492.825
3,11355.226,208.515,263.214,0.000,157.682,3142.826,3142.826,610.581,3142.826,642.380,...,3657.314,3877.335,3962.277,3956.370,3792.478,3962.277,4219.022,4652.248,3372.586,3445.236
4,11367.751,50.833,105.532,296.900,0.000,2985.144,2985.144,452.899,2985.144,654.905,...,3590.118,3810.139,3895.081,3889.174,3725.282,3895.081,4151.826,4585.052,3308.216,3380.866


In [247]:
distance.columns = distance.columns.astype('int')

In [248]:
# CW

clients = 811                                                  # 客户数量
capacity = 300                                              # 车辆载重                                        # 车辆一次行驶的最大距离                                             # 各个客户及配送中心距离
demand = van.Demand.values                                # 811个客户分布需要的货物的需求量，第0位为配送中心
savings = []                                               # 节约度
Routes = []                                                # 路线
Cost = 0  

In [249]:
# itter 0 routes
Routes=[]
for i in range(1, len(demand)):
    Routes.append([i])

# caculate the saving, neglect deport
savings = []
for i in range(1, len(Routes)+1):
    for j in range(1, len(Routes)+1):
        if i==j:
            pass
        else:
            saving = (distance[i][0] + distance[0][j]) - distance[i][j]
            savings.append([i, j, saving])
savings = sorted(savings, key=itemgetter(2), reverse=True)

In [250]:
savings 

[[301, 302, 24327.908000000003],
 [302, 301, 24327.908000000003],
 [290, 301, 24060.978000000003],
 [290, 302, 24060.978000000003],
 [301, 290, 24060.978000000003],
 [302, 290, 24060.978000000003],
 [305, 292, 24037.408000000003],
 [305, 301, 24036.96],
 [305, 302, 24036.96],
 [301, 292, 24036.559],
 [302, 292, 24036.559],
 [307, 305, 23928.759],
 [305, 307, 23918.593],
 [290, 289, 23902.160000000003],
 [301, 289, 23902.160000000003],
 [302, 289, 23902.160000000003],
 [289, 301, 23902.16],
 [289, 302, 23902.16],
 [289, 290, 23902.159999999996],
 [290, 288, 23901.737],
 [301, 288, 23901.737],
 [302, 288, 23901.737],
 [289, 288, 23901.736999999994],
 [290, 292, 23878.381],
 [304, 307, 23869.206000000006],
 [304, 305, 23869.206000000002],
 [306, 292, 23770.635000000002],
 [308, 292, 23770.635000000002],
 [306, 305, 23770.635],
 [306, 307, 23770.635],
 [307, 292, 23770.635],
 [307, 306, 23770.635],
 [307, 308, 23770.635],
 [308, 305, 23770.635],
 [308, 307, 23770.635],
 [306, 301, 23770.18

In [251]:

for i in range(len(savings)):
            startRoute = []
            endRoute = []
            routeDemand = 0
            for j in range(len(Routes)):
                if (savings[i][0] == Routes[j][-1]):
                    endRoute = Routes[j]
                elif (savings[i][1] == Routes[j][0]):
                    startRoute = Routes[j]
                
                if ((len(startRoute) != 0) and (len(endRoute) != 0)):
                    for k in range(len(startRoute)):
                        routeDemand += demand[startRoute[k]]
                    for k in range(len(endRoute)):
                        routeDemand += demand[endRoute[k]]
                    routeDistance = 0
                    routestore = [0]+endRoute+startRoute+[0]
                    for i in range(len(routestore)-1):
                        # print(routestore[i],routestore[i+1])
                        # print(self.distance[routestore[i]][routestore[i+1]])
                        routeDistance += distance[routestore[i]][routestore[i+1]]

                    #print(routestore,"== ==:",routeDistance)

                    if (routeDemand <= capacity):     # 按照限制规则对​​路线进行更改
                        Routes.remove(startRoute)
                        Routes.remove(endRoute)
                        Routes.append(endRoute + startRoute)
                    break
            

In [252]:
# insert deport
for i in range(len(Routes)):
    Routes[i].insert(0,0)
    Routes[i].insert(len(Routes[i]),0)

In [253]:
len(Routes)

23

In [254]:
dd=[]
df=pd.DataFrame()
for i in Routes:
            costs = 0
            load = 0
            for j in range(len(i)-1):
                
                load += demand[i[j]]
                costs += distance[i[j]][i[j+1]]
                
            df = df.append(pd.DataFrame({'Route':[i],'Load':[load],'distance':[costs]}))
                
            print ('Route:',i)
            print ('Load:',load)
            print ('Distance:',costs)
            print('---'*10)
            dd.append(costs)
Cost = 0
for i in range(len(Routes)):
            for j in range(len(Routes[i]) - 1):
                Cost += distance[Routes[i][j]][Routes[i][j + 1]]
            
print('Total Distance:', Cost)

Route: [0, 567, 459, 434, 360, 357, 356, 358, 353, 349, 352, 342, 340, 339, 410, 429, 355, 344, 287, 286, 328, 331, 333, 335, 445, 446, 336, 442, 323, 334, 327, 332, 330, 329, 320, 321, 322, 311, 298, 297, 326, 324, 319, 276, 275, 407, 403, 388, 382, 401, 425, 418, 422, 411, 0]
Load: 298.0
Distance: 32193.26999999999
------------------------------
Route: [0, 186, 58, 55, 57, 53, 59, 60, 45, 40, 56, 30, 46, 47, 48, 49, 50, 51, 52, 264, 236, 266, 254, 255, 235, 243, 244, 249, 272, 273, 262, 367, 348, 350, 341, 346, 347, 345, 472, 0]
Load: 300.0
Distance: 25511.12800000001
------------------------------
Route: [0, 112, 105, 98, 149, 152, 158, 162, 103, 102, 100, 160, 99, 95, 101, 92, 93, 94, 147, 97, 104, 106, 81, 82, 109, 110, 185, 108, 184, 183, 170, 203, 204, 207, 192, 0]
Load: 298.0
Distance: 25143.467999999993
------------------------------
Route: [0, 672, 167, 163, 168, 169, 161, 113, 96, 85, 91, 90, 146, 140, 141, 134, 130, 133, 138, 137, 132, 139, 145, 762, 153, 763, 148, 154, 155

In [255]:
df

,Route,Load,distance
0,"[0, 567, 459, 434, 360, 357, 356, 358, 353, 34...",298.0,32193.270
0,"[0, 186, 58, 55, 57, 53, 59, 60, 45, 40, 56, 3...",300.0,25511.128
0,"[0, 112, 105, 98, 149, 152, 158, 162, 103, 102...",298.0,25143.468
0,"[0, 672, 167, 163, 168, 169, 161, 113, 96, 85,...",300.0,24629.928
0,"[0, 714, 717, 724, 725, 719, 734, 723, 680, 67...",298.0,21328.238
0,"[0, 501, 73, 38, 34, 29, 39, 28, 26, 36, 37, 4...",300.0,26970.529
0,"[0, 502, 217, 216, 79, 80, 77, 86, 87, 84, 83,...",299.0,30569.569
0,"[0, 508, 176, 187, 181, 188, 189, 190, 208, 22...",300.0,21627.072
0,"[0, 811, 697, 810, 694, 770, 772, 773, 775, 76...",299.0,21652.227
0,"[0, 509, 177, 193, 194, 198, 199, 493, 499, 49...",300.0,22283.433


In [256]:
df.to_csv('./CW_Van_Route_1219.csv')